# Data Collection

To analyze the flights' delay causes, we need two parts of data, the information on flights and the external factors. The flights information could be downloaded on the website https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr. Then, we choose origin city and destination city weather as the external factors to help analyze the flights' delays, https://openweathermap.org/ provides convenient APIs to get the historical weather data.

## Get The Flight Dataset

The flight dataset was downloaded manually and saved in the directory `dataset`. The data is so large that the reading of CSV files is slow, we use sqlite3 organize these data to reach a faster query speed. Sqlite3 is a light database based on local files which need not servers or executing clients. 

First, I create a database named `flight.db`, all such settings are stored in a JSON format file named `config.json` which make the changing of settings more convenient. `Python` provides `json` package for JSON files, by which the config in the file would be loaded as a dictionary or a list.

In [1]:
import sqlite3  # sqlite3 provides a lightweight disk-based database that doesn’t require a separate server process
import pandas as pd
import json

# import the config setting
with open('config.json','r+',encoding='utf-8') as f:
    config=json.load(f)

# connect the database
db=sqlite3.connect(config['Database'])
print('[INFO]: connect database successfully')

# create a cursor object to operate the db
cur=db.cursor()

[INFO]: connect database successfully


Before writing table to the database, we should check out whether the table exists.

In [2]:
def get_tables(cur):
    """get all the table name in the database

    Args:
    ----------
        cur (object): the cursor object of the database
    
    Return:
    ----------
    List
        A list contains all the table name in the database
    """
    try:
        cur.execute("select name from sqlite_master where type='table'")
        return cur.fetchall()
    except Exception as e:
        print(e)

def table_exist(table_name:str)->bool:
    """if the table exists in the database

    Args:
        table_name (str): the tal

    Returns:
        bool: True if the table exists in the database else False
    """
    tables=get_tables(cur)

    # judge if the flight table exists in the db
    exist=False
    for table in tables:
        if table_name in table:
            exist=True
    return exist

If the table exists, we could skip the procedure of writing data to database and jump to the next step. Or we need to read `csv` files and write the table to database. Similarly, the filepath list is saved in the config files, you could change it to writing any data you want to the database.

In [3]:
# read csv files and write the data to sql database if the table not exists
if not table_exist('flight'):
    for filepath in config['Daily Dataset Path List']:
        df=pd.read_csv(filepath)
        df.to_sql("flight",db,if_exists='append')
        print(f'File {filepath} has been wrote to the sqlite database')

File dataset/T_ONTIME_MARKETING_01.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_02.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_03.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_04.csv has been wrote to the sqlite database


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


File dataset/T_ONTIME_MARKETING_05.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_06.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_07.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_08.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_09.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_10.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_11.csv has been wrote to the sqlite database
File dataset/T_ONTIME_MARKETING_12.csv has been wrote to the sqlite database


## Get The Weather Data
Then we need to get the weather data of the cities. First, cities should be transformed to their longitude and latitude. Then call the historical weather API with the longitude and the latitude could receive a JSON response containing daily weather data of cities last year. Finally, the JSON response could be transformed to DataFrame and export to the sql database. 

First, we import `requests` packages for sending requests to websites, and other packages to assist this procedure. Then create a `client` to ensure the continuity of web requesting. 

In [4]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

MAX_RETRIES = 10    # Number of times to retry a request
SLEEP_INTERVAL = 0.1    # sleep interval to retry

retries=Retry(
    total=MAX_RETRIES,
    backoff_factor=SLEEP_INTERVAL,
    status_forcelist=[403, 500, 502, 503, 504],
)

# create client to send requests
client = requests.Session()
client.mount("http://", HTTPAdapter(max_retries=retries))
client.mount("https://", HTTPAdapter(max_retries=retries))


The city name should be transformed to longitude and latitude for the weather fetching, so I make a function achieving this procedure.

In [5]:
def city2cordinate(city_name:str)->dict:
    """transform city to longitude and latitude

    Args:
    ----------
        city_name (str): the city to be transformed
    
    Return:
    ----------
        dict: A dictionary contains longitude and latitude of the city
    """
    params={
        "q":city_name,
        "limit":5,
        "appid":config['API Key']
    }
    # send a get request with params and receive a json format response
    response=client.get(config['Geocoding API'],params=params).json()
    
    """
    the response includes serval cities from different country,
    choose the city in the US and return its cordinate.
    """
    for item in response:
        if item['country']=='US':
            return {'lon':item['lon'],
                    'lat':item['lat']}
    else:
        print(city_name,response)
        return False

# test the function city_name2lon_lat
cordinate=city2cordinate('San Antonio')
print(cordinate)

{'lon': -98.4951405, 'lat': 29.4246002}


Similar to the weather API, the `get weather` function get the longitude and latitude at the location and return the last year weather data.

In [7]:
def get_weather(lon:float,lat:float)->dict:
    """get the last-year weather data by longitude and latitude at the location 

    Args:
    ----------
        lon (float): longitude of the location
        lat (float): latitude of the location
    
    Return:
    ----------
        dict: A json dictionary contains the weather data at the location
    """
    params={
        "lon":lon,
        "lat":lat,
        "appid":config['API Key'],
    }
    response=client.get(config['Weather Data API'],params=params).json()
    if response['cod']==200:
        return response['result']
    else:
        return ValueError()

# test the function get_weather
weather_info=get_weather(**cordinate)
print(weather_info[0])


{'month': 1, 'day': 1, 'temp': {'record_min': 270.58, 'record_max': 299.6, 'average_min': 279.68, 'average_max': 287.04, 'median': 282.89, 'mean': 282.48, 'p25': 276.75, 'p75': 285.63, 'st_dev': 7.03, 'num': 240}, 'pressure': {'min': 967, 'max': 1040, 'median': 1019.5, 'mean': 1019.42, 'p25': 1013, 'p75': 1026, 'st_dev': 9.76, 'num': 238}, 'humidity': {'min': 26, 'max': 100, 'median': 76, 'mean': 73.75, 'p25': 62, 'p75': 87, 'st_dev': 16.65, 'num': 238}, 'wind': {'min': 0, 'max': 8.7, 'median': 3.1, 'mean': 3.22, 'p25': 1.58, 'p75': 4.6, 'st_dev': 1.94, 'num': 240}, 'precipitation': {'min': 0, 'max': 0.3, 'median': 0, 'mean': 0.06, 'p25': 0, 'p75': 0, 'st_dev': 0.12, 'num': 240}, 'clouds': {'min': 0, 'max': 100, 'median': 90, 'mean': 58.3, 'p25': 1, 'p75': 90, 'st_dev': 39.88, 'num': 240}}


To writing the weather data to database, the weather data response should be transformed from JSON format to `pd.DataFrame` format. 

In [8]:
def extract_weather(weather_info:dict,city_name:str)->pd.DataFrame:
    """ extract weather data from a dict and convert it to a dataframe

    Args:
    ----------
        weather_info (list(dict)): 
        example:
        [{
            "month": 1,
            "day": 1,
            "temp": {
                "record_min": 263.94,
                "record_max": 291.59,
                "average_min": 271.7,
                "average_max": 283.32,
                "median": 276.94,
                "mean": 277.75,
                "p25": 273.41,
                "p75": 282.86,
                "st_dev": 6.98,
                "num": 240
            },
            "pressure": {
                "min": 913,
                "max": 1026,
                "median": 1012,
                "mean": 1002.49,
                "p25": 996.5,
                "p75": 1020.5,
                "st_dev": 27.1,
                "num": 240
            },
            "humidity": {
                "min": 19,
                "max": 100,
                "median": 71,
                "mean": 69.72,
                "p25": 53,
                "p75": 99,
                "st_dev": 25.28,
                "num": 240
            },
            "wind": {
                "min": 0,
                "max": 9.3,
                "median": 1.21,
                "mean": 1.71,
                "p25": 0.89,
                "p75": 2.21,
                "st_dev": 1.49,
                "num": 240
            },
            "precipitation": {
                "min": 0,
                "max": 0.3,
                "median": 0,
                "mean": 0,
                "p25": 0,
                "p75": 0,
                "st_dev": 0.03,
                "num": 240
            },
            "clouds": {
                "min": 0,
                "max": 99,
                "median": 1,
                "mean": 24,
                "p25": 0,
                "p75": 40,
                "st_dev": 32.79,
                "num": 240
            }
        }]
    
    Return
    ----------
        pd.DataFrame: A dataframe contains the weather data converted from the input dict
    """
    # generate the columns of dataframe
    columns=['city_name','month','day']
    factors=['temp','pressure','humidity','wind','precipitation','clouds']
    infos=['min','max','mean','p25','p75','median','st_dev']
    for factor in factors:
        for info in infos:
            columns.append(f'{factor}_{info}') 
    # create an empty df
    df=pd.DataFrame(columns=columns)
    for item in weather_info:
        row={}
        row['city_name']=city_name
        row['month']=item['month']
        row['day']=item['day']
        for factor in factors:
            for info in infos:
                # extracting tempure min and max is special  
                if factor=='temp':
                    if info=='min':
                        row[f'{factor}_{info}']=item[factor]['average_min']
                        continue
                    if info=='max':
                        row[f'{factor}_{info}']=item[factor]['average_max']
                        continue
                row[f'{factor}_{info}']=item[factor][info]
        df=df.append(row,ignore_index=True)
    return df

# test the function extract_weather
df=extract_weather(weather_info,'San Antonio')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   city_name             366 non-null    object 
 1   month                 366 non-null    object 
 2   day                   366 non-null    object 
 3   temp_min              366 non-null    float64
 4   temp_max              366 non-null    float64
 5   temp_mean             366 non-null    float64
 6   temp_p25              366 non-null    float64
 7   temp_p75              366 non-null    float64
 8   temp_median           366 non-null    float64
 9   temp_st_dev           366 non-null    float64
 10  pressure_min          366 non-null    object 
 11  pressure_max          366 non-null    object 
 12  pressure_mean         366 non-null    float64
 13  pressure_p25          366 non-null    object 
 14  pressure_p75          366 non-null    object 
 15  pressure_median       3

Now, We could extract last-year weather data of all the cities in the flight data. First, we execute a `select` command fetching the set of cites from flight table. Then, the historical weather data could be extracted and transformed to a table by the above function. Finally, the weather data could be appended to the `weather` table in the `flight.db` database. Similarly, we should check whether the weather table exists before writing data, which could avoid repeating to write. If you would like to update dataset, you could delete the table and rerun this block.

In [9]:
# store historical weather data of all cities in the database
# get distinct cities from table flight
cur.execute('SELECT DISTINCT ORIGIN_CITY_NAME FROM flight')
city_list=cur.fetchall()

if not table_exist('weather'):

    # acquire weather for each city
    # write these weather into database
    for city, in city_list:
        # draw the entire city name
        real_city=city[:city.find(',')]
        if '/' in real_city:
            real_city=real_city[:real_city.find('/')]
        print(f'[INFO] downloading historical weather data of city: {real_city}')
        # get longitude and latitude
        cordinate=city2cordinate(real_city)
        if not cordinate:
            continue
        # crawl historical weather from api
        weather_info=get_weather(**cordinate)
        # transform json to df format
        weather_df=extract_weather(weather_info,city)
        # write df to database
        weather_df.to_sql('weather',db,if_exists='append')
    